# Exploring how ETL will be done.
This notebook will investigate the viability of transforming the data into parquet files using Spark.
Pros with using Parquet over csv are that it has an embedded schema, takes up less storage and can extract individual columns much faster.

Note that **this notebook is not an EDA**. The goal is for ETL design only therefore EDA is out of scope.

This notebook will: see what data we have, understand the schemas and determine how to move forward with the ETL script.

## Discover what datasets we have.
Start by defining some folder paths and looking at the downloaded data.

In [1]:
from config import proj
from pathlib import Path
import zipfile
import os
import py7zr # need for decompression of 7z
import time # for timing execution

PATH_RAW_DATA_DIR = Path(proj.proj_paths["top"]).joinpath('data').joinpath('raw')
PATH_INTERIM_DATA_DIR = Path(proj.proj_paths["top"]).joinpath('data').joinpath('interim')
PATH_PROC_DATA_DIR = Path(proj.proj_paths["top"]).joinpath('data').joinpath('processed')

In [7]:
# Get compressed file name
print(os.listdir(PATH_RAW_DATA_DIR)[0])

favorita-grocery-sales-forecasting.zip


In [8]:
PATH_RAW_DATA = PATH_RAW_DATA_DIR.joinpath('favorita-grocery-sales-forecasting.zip')

In [13]:
# Check contents of compressed file
with zipfile.ZipFile(PATH_RAW_DATA, 'r') as zip_ref:
    for file in zip_ref.infolist():
        print(file)

<ZipInfo filename='holidays_events.csv.7z' compress_type=deflate file_size=1898 compress_size=1903>
<ZipInfo filename='items.csv.7z' compress_type=deflate file_size=14315 compress_size=14320>
<ZipInfo filename='oil.csv.7z' compress_type=deflate file_size=3762 compress_size=3767>
<ZipInfo filename='sample_submission.csv.7z' compress_type=deflate file_size=666528 compress_size=649511>
<ZipInfo filename='stores.csv.7z' compress_type=deflate file_size=648 compress_size=653>
<ZipInfo filename='test.csv.7z' compress_type=deflate file_size=4885065 compress_size=4886553>
<ZipInfo filename='train.csv.7z' compress_type=deflate file_size=474092593 compress_size=474237203>
<ZipInfo filename='transactions.csv.7z' compress_type=deflate file_size=219499 compress_size=219569>


All the contents are also compressed. Will start with the largest one and generate a parquet file with it and do ensure everything works smoothly.

In [14]:
# Unzip master file
with zipfile.ZipFile(PATH_RAW_DATA, 'r') as zip_ref:
    zip_ref.extractall(PATH_INTERIM_DATA_DIR)

In [24]:
# Get file names of interim
file_list = os.listdir(PATH_INTERIM_DATA_DIR)
for file in file_list:
    print(file + ' - ' + str(os.stat(PATH_INTERIM_DATA_DIR.joinpath(file)).st_size))

items.csv.7z - 14315
transactions.csv.7z - 219499
holidays_events.csv.7z - 1898
train.csv.7z - 474092593
stores.csv.7z - 648
oil.csv.7z - 3762
test.csv.7z - 4885065
sample_submission.csv.7z - 666528


train.csv.7z is clearly the largest file.
stores.csv7z is the smallest.

Will start small with the stores, then work with train.

## Viability of parquet and Pandas API with small dataset (Stores data).
Stores is the smallest dataset. Will look at working with Pandas API and parquet files.

In [33]:
# Unzip stores
PATH_INTERIM_DATA_STORES = PATH_INTERIM_DATA_DIR.joinpath('stores.csv.7z')

with py7zr.SevenZipFile(PATH_INTERIM_DATA_STORES, 'r') as zip_ref:
    zip_ref.extractall(PATH_INTERIM_DATA_DIR)

In [40]:
print('stores.csv' in os.listdir(PATH_INTERIM_DATA_DIR))

True


Can see stores.csv is in the folder.

Now we will convert into a parquet file using Spark.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [51]:
# Show top 5 rows
df = spark.read.csv(str(PATH_INTERIM_DATA_DIR.joinpath('stores.csv')), header=True)
df.show(5)

+---------+-------------+--------------------+----+-------+
|store_nbr|         city|               state|type|cluster|
+---------+-------------+--------------------+----+-------+
|        1|        Quito|           Pichincha|   D|     13|
|        2|        Quito|           Pichincha|   D|     13|
|        3|        Quito|           Pichincha|   D|      8|
|        4|        Quito|           Pichincha|   D|      9|
|        5|Santo Domingo|Santo Domingo de ...|   D|      4|
+---------+-------------+--------------------+----+-------+
only showing top 5 rows



In [69]:
# Update schema
df.printSchema()

root
 |-- store_nbr: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- type: string (nullable = true)
 |-- cluster: string (nullable = true)



Schema just defaults everything to string which is not useful. Will need to clarify what the schema will be. This is be the case for all datasets most likely.

In [72]:
from pyspark.sql.types import StructType, IntegerType, StringType

# Need to clarify the schema
stores_schema = StructType()\
    .add('store_nbr', IntegerType(), True)\
    .add('city', StringType(), True)\
    .add('state', StringType(), True)\
    .add('type', StringType(), True)\
    .add('cluster', StringType(), True) # is an integer, but given it's a grouping better to keep it categorical

In [76]:
# Rather than cast everything in place, we will just read in the file again
df = spark.read.csv(str(PATH_INTERIM_DATA_DIR.joinpath('stores.csv')), header=True, schema=stores_schema)
df.printSchema()
df.show(5)

root
 |-- store_nbr: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- type: string (nullable = true)
 |-- cluster: string (nullable = true)

+---------+-------------+--------------------+----+-------+
|store_nbr|         city|               state|type|cluster|
+---------+-------------+--------------------+----+-------+
|        1|        Quito|           Pichincha|   D|     13|
|        2|        Quito|           Pichincha|   D|     13|
|        3|        Quito|           Pichincha|   D|      8|
|        4|        Quito|           Pichincha|   D|      9|
|        5|Santo Domingo|Santo Domingo de ...|   D|      4|
+---------+-------------+--------------------+----+-------+
only showing top 5 rows



In [86]:
# Write parquet file
PATH_STORES_PQ = str(PATH_PROC_DATA_DIR.joinpath('stores.parquet'))
df.write.parquet(PATH_STORES_PQ)

In [79]:
# Read in from parquet
df = spark.read.parquet(PATH_STORES_PQ)
df.printSchema()
df.show(6)

root
 |-- store_nbr: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- type: string (nullable = true)
 |-- cluster: string (nullable = true)

+---------+-------------+--------------------+----+-------+
|store_nbr|         city|               state|type|cluster|
+---------+-------------+--------------------+----+-------+
|        1|        Quito|           Pichincha|   D|     13|
|        2|        Quito|           Pichincha|   D|     13|
|        3|        Quito|           Pichincha|   D|      8|
|        4|        Quito|           Pichincha|   D|      9|
|        5|Santo Domingo|Santo Domingo de ...|   D|      4|
|        6|        Quito|           Pichincha|   D|     13|
+---------+-------------+--------------------+----+-------+
only showing top 6 rows



Schema looks good. We could also partition this file but wont have to since it is so small.

### Pandas API with stores data
Given pandas is so commonly used with Python, the pandas api on spark will be used moving forward.


In [82]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
psdf = df.pandas_api()
psdf.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [84]:
psdf.describe()

,store_nbr
count,54.000000
mean,27.500000
std,15.732133
min,1.000000
25%,14.000000
50%,27.000000
75%,41.000000
max,54.000000


Above looks good and works well.

Below we will do the same for the largest file (train.csv) and make sure to partition it.

## Viability of parquet and Pandas API with large data (train data).
The train data is large so will likely be a bit fiddly in terms of memory.
The below cells will attempt to convert the train data to a parquet file and

In [3]:
# Unzip train
# Note: This is a bit duplicated at the moment, but will clean it up in the main etl script
PATH_INTERIM_DATA_TRAIN = PATH_INTERIM_DATA_DIR.joinpath('train.csv.7z')

with py7zr.SevenZipFile(PATH_INTERIM_DATA_TRAIN, 'r') as zip_ref:
    zip_ref.extractall(PATH_INTERIM_DATA_DIR)

In [3]:
start_time = time.time()

df = spark.read.csv(str(PATH_INTERIM_DATA_DIR.joinpath('train.csv')), header=True)

print("%s seconds" % round((time.time() - start_time), 2))

6.03 seconds


In [8]:
df.show(5)

+---+----------+---------+--------+----------+-----------+
| id|      date|store_nbr|item_nbr|unit_sales|onpromotion|
+---+----------+---------+--------+----------+-----------+
|  0|2013-01-01|       25|  103665|       7.0|       null|
|  1|2013-01-01|       25|  105574|       1.0|       null|
|  2|2013-01-01|       25|  105575|       2.0|       null|
|  3|2013-01-01|       25|  108079|       1.0|       null|
|  4|2013-01-01|       25|  108701|       1.0|       null|
+---+----------+---------+--------+----------+-----------+
only showing top 5 rows



In [ ]:
# psdf = df.pandas_api()
# psdf.head()
# Pandas API is taking so long compared to Spark itself? head() must collect the entire dataframe.

Very quick to read in and show the head using spark without pandas api. A column view will likely be very slow given the underlying data is a csv.
Need to understand the data a bit better so will split up into chunks and describe. Running all at once will likely run out of memory on this machine.

In [ ]:
psdf = df.pandas_api()

In [12]:
print(len(psdf.index))

125497040


Takes a long time to even get the number of rows which is 125m. Will definitely need to partition this. Working with csv will take a long time. Will write as parquet for now, then do more exploration and update it.
Basically we want to know
- what to make the structure
- how to partition

In [4]:
PATH_TRAIN_INTERIM_PQ = str(PATH_INTERIM_DATA_DIR.joinpath('train.parquet'))

In [ ]:
df.write.parquet(PATH_TRAIN_INTERIM_PQ)

In [5]:
# Read in from parquet
df = spark.read.parquet(PATH_TRAIN_INTERIM_PQ)

In [6]:
df.printSchema()
df.show(5)

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- store_nbr: string (nullable = true)
 |-- item_nbr: string (nullable = true)
 |-- unit_sales: string (nullable = true)
 |-- onpromotion: string (nullable = true)

+---+----------+---------+--------+----------+-----------+
| id|      date|store_nbr|item_nbr|unit_sales|onpromotion|
+---+----------+---------+--------+----------+-----------+
|  0|2013-01-01|       25|  103665|       7.0|       null|
|  1|2013-01-01|       25|  105574|       1.0|       null|
|  2|2013-01-01|       25|  105575|       2.0|       null|
|  3|2013-01-01|       25|  108079|       1.0|       null|
|  4|2013-01-01|       25|  108701|       1.0|       null|
+---+----------+---------+--------+----------+-----------+
only showing top 5 rows



In [ ]:
# psdf = df.pandas_api()

In [ ]:
# psdf['onpromotion'].iloc[1:10]

After some more research can see that many are having problems with the speed of the Pandas api for spark. Will still with spark functions.

In [ ]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
PATH_TRAIN_INTERIM_PQ = str(PATH_INTERIM_DATA_DIR.joinpath('train.parquet'))

In [ ]:
df = spark.read.parquet(PATH_TRAIN_INTERIM_PQ)

In [42]:
df.select('onpromotion').show(5)
# so much faster!

+-----------+
|onpromotion|
+-----------+
|       null|
|       null|
|       null|
|       null|
|       null|
+-----------+
only showing top 5 rows



In [13]:
df.select('onpromotion').distinct().collect()

[Row(onpromotion=None), Row(onpromotion='False'), Row(onpromotion='True')]

In [8]:
df = df.withColumn("id",df.id.cast('int'))\
    .withColumn("date",df.date.cast('date'))\
    .withColumn("store_nbr",df.store_nbr.cast('int'))\
    .withColumn("item_nbr",df.item_nbr.cast('int'))\
    .withColumn("unit_sales",df.unit_sales.cast('float'))\
    .withColumn("onpromotion",df.onpromotion.cast('boolean'))
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- store_nbr: integer (nullable = true)
 |-- item_nbr: integer (nullable = true)
 |-- unit_sales: float (nullable = true)
 |-- onpromotion: boolean (nullable = true)



ID column looks to be useless and just a row number we can probably remove.

In [14]:
print(df.select(df.id).take(5))
print(df.select(df.id).tail(5))
df.count()
# Selecting distinct ran out of memory, but can see the head and tail look like row nums.

[Row(id='0'), Row(id='1'), Row(id='2'), Row(id='3'), Row(id='4')]


[Row(id='125497035'), Row(id='125497036'), Row(id='125497037'), Row(id='125497038'), Row(id='125497039')]


125497040

In [15]:
df = df.drop('id')
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- store_nbr: string (nullable = true)
 |-- item_nbr: string (nullable = true)
 |-- unit_sales: string (nullable = true)
 |-- onpromotion: string (nullable = true)



In [38]:
# Look at a single item and see if promo has an impact
# this is just an eye-balling, proper analysis will be done during eda
df.filter(df.item_nbr == 103665)\
    .filter(df.onpromotion.isNotNull())\
    .groupby(df.store_nbr, df.onpromotion)\
    .avg('unit_sales')\
    .sort(df.store_nbr, df.onpromotion)\
    .show()

+---------+-----------+------------------+
|store_nbr|onpromotion|   avg(unit_sales)|
+---------+-----------+------------------+
|        1|      false|3.5494276795005204|
|        1|       true|              4.04|
|        2|      false|3.1278493557978195|
|        2|       true|3.6206896551724137|
|        3|      false| 4.580110497237569|
|        3|       true|           4.40625|
|        4|      false| 3.128526645768025|
|        4|       true|3.6666666666666665|
|        5|      false| 6.432795698924731|
|        5|       true| 7.368421052631579|
|        6|      false| 4.289800995024875|
|        6|       true|2.4545454545454546|
|        7|      false| 5.587735849056604|
|        7|       true| 6.428571428571429|
|        8|      false| 2.788482834994463|
|        8|       true|3.0434782608695654|
|        9|      false| 3.294940796555436|
|        9|       true| 3.937007874015748|
|       11|      false| 2.387862796833773|
|       11|       true|2.2660550458715596|
+---------+

Operations seem to be possible without using the Pandas API and leveraging what PySpark has.
Will perform some more operations on the larger dataset to ensure this will be usable.
Saying that, it's likely we will not have to use all the data to get a decent model. But useful to be able to view it all easily during the EDA.

In [49]:
# Take the average of every item store and collect. See how long it takes and if it's reasonable for EDA.
start_time = time.time()

df.groupby(df.item_nbr, df.store_nbr)\
    .avg('unit_sales')\
    .collect()

print("%s seconds" % round((time.time() - start_time), 2))

29.03 seconds


30 seconds for the average sales of every item-store is pretty good (given how old my computer is). This will be fine for EDA and mostly likely ok for model training.
But as mentioned above, we will not have to use all the data for a reasonable model.

### Partitioning train data
Given it is time series data, it would likely benefit by being paritioned by date. This would make it very easy to add new data, filter out old data, explore spikes, etc.

### Train schema

In [ ]:
from pyspark.sql.types import StructType, IntegerType, StringType, DateType, FloatType, BooleanType

schema = StructType()\
    .add('date', DateType(), True)\
    .add('store_nbr', IntegerType(), True)\
    .add('item_nbr', IntegerType(), True)\
    .add('unit_sales', FloatType(), True)\
    .add('onpromotion', BooleanType(), True)

## Determine schemas for remaining datasets
Will put the results into `src/data/schemas`